# Use Customer_Segmentation Notebook template

In [2]:
#Snowpark lib
from snowflake.snowpark import Session

# Data Science Libs
import numpy as np
import pandas as pd

# create_temp_table warning suppresion
import warnings; warnings.simplefilter('ignore')

#ConfigParser to read ini file
import configparser

# Different ways to achieve Snowflake Connection

## 1. Code to establish connection using Config Parser and .ini file

In [3]:
config = configparser.ConfigParser()
config.read("credentials.ini")

['credentials.ini']

In [4]:
connection_parameters = {
    "user": f'{config["Snowflake"]["user"]}',
    "password": os.getenv('Snowflake_password'),
    "account": f'{config["Snowflake"]["account"]}',
    "WAREHOUSE": f'{config["Snowflake"]["WAREHOUSE"]}',
    "DATABASE": f'{config["Snowflake"]["DATABASE"]}',
    "SCHEMA": f'{config["Snowflake"]["SCHEMA"]}'
}

In [4]:
def snowflake_connector(conn):
    try:
        session = Session.builder.configs(conn).create()
        print("connection successful!")
    except:
        raise ValueError("error while connecting with db")
    return session

session = snowflake_connector(connection_parameters)

connection successful!


In [5]:
df = session.table("CUSTOMER_DATA_INSIGHT_TRAIN_OUTPUT")

In [6]:
df = df.to_pandas()

In [7]:
df.tail()

,CUSTOMER_ID,DATE,AGE,SEX,EDUCATION_LEVEL,EMPLOYMENT_STATUS,HOBBIES,MARITAL_STATUS,DEPENDENTS,REGISTRATION_DATE,...,PURCHASES_INSTALLMENTS_FREQUENCY,PURCHASES_TRX,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,CLUSTER,CLUSTER_TYPE,PREDICTION,REPORT_DATE,CLUSTER_ID
79995,01d68689-06f0-4d02-94e9-e860dc631c39,2022-04-01,33,MALE,College,Unemployed,chess,Single,One Dependent,2000-12-12,...,Very low,15.12,1858.996274,198.293611,0.0,0,Conservative Spenders,0,2022-04-01,CLUSTER_0
79996,f51a24f1-cacc-4637-813b-aef52bbb8d16,2023-11-01,33,FEMALE,MD,Unemployed,video-games,Single,One Dependent,2021-04-12,...,Very low,13.51,1929.815180,202.604342,0.0,0,Conservative Spenders,0,2023-11-01,CLUSTER_0
79997,83c3a4de-4346-47b0-bc8b-4171d72c68e0,2024-03-01,22,FEMALE,MD,Unemployed,movies,Single,No Dependents,1997-09-12,...,High,33.00,339.702306,175.285790,0.0,0,Conservative Spenders,0,2024-03-01,CLUSTER_0
79998,f741d69b-d623-4f7e-9cd1-01bf5c3b03fd,2023-03-01,22,MALE,Masters,Unemployed,movies,Single,No Dependents,2014-01-12,...,High,30.69,360.084444,187.555795,0.0,0,Conservative Spenders,0,2023-03-01,CLUSTER_0
79999,a892b7ae-4677-40a4-9771-c612892ec24c,2023-01-01,22,MALE,JD,Unemployed,chess,Single,No Dependents,2015-03-12,...,High,30.36,351.591887,161.262927,0.0,0,Conservative Spenders,0,2023-01-01,CLUSTER_0


In [8]:
df.shape

(80000, 32)

In [9]:
df.columns

Index(['CUSTOMER_ID', 'DATE', 'AGE', 'SEX', 'EDUCATION_LEVEL',
       'EMPLOYMENT_STATUS', 'HOBBIES', 'MARITAL_STATUS', 'DEPENDENTS',
       'REGISTRATION_DATE', 'BANK_ACCOUNT_TYPE', 'BALANCE',
       'BALANCE_FREQUENCY', 'CREDIT_LIMIT', 'CASH_ADVANCE',
       'CASH_ADVANCE_FREQUENCY', 'CASH_ADVANCE_TRX', 'PURCHASES',
       'ONEOFF_PURCHASES', 'INSTALLMENTS_PURCHASES', 'PURCHASES_FREQUENCY',
       'ONEOFF_PURCHASES_FREQUENCY', 'PURCHASES_INSTALLMENTS_FREQUENCY',
       'PURCHASES_TRX', 'PAYMENTS', 'MINIMUM_PAYMENTS', 'PRC_FULL_PAYMENT',
       'CLUSTER', 'CLUSTER_TYPE', 'PREDICTION', 'REPORT_DATE', 'CLUSTER_ID'],
      dtype='object')

In [10]:
to_retain = ['CUSTOMER_ID', 'AGE', 'EMPLOYMENT_STATUS', 'HOBBIES', 'MARITAL_STATUS', 
       'BANK_ACCOUNT_TYPE', 'BALANCE','CREDIT_LIMIT', 'CASH_ADVANCE',
       'PURCHASES','INSTALLMENTS_PURCHASES','PAYMENTS', 'CLUSTER_TYPE']

In [11]:
new_df = df[to_retain]

In [12]:
new_df.shape

(80000, 13)

In [13]:
new_df.to_csv('/data/train_df.csv',index=False)

### Write Back logic ####

In [5]:
cc_df = pd.read_csv('/data/creditcards.csv')

In [6]:
cc_df.head()

,CARDID,CARDNAME,INTERESTRATE,CREDITLIMIT,ANNUALFEE,REWARDSPROGRAM,INTRODUCTORYAPR,FOREIGNTRANSACTIONFEE,LATEPAYMENTFEE,BALANCETRANSFERFEE,CASHADVANCEFEE,SECURITYDEPOSIT,CARDTYPE
0,1,Admin One SavorOne Cash Rewards Credit Card,15.49,6000,0,Bills and Utilities,0,0.0,39,3,10,0,Unsecured
1,2,Recover Cash bank card,13.99,6000,95,Bills and Utilities,0,2.7,39,5,5,0,Unsecured
2,3,City Double Cash Card,13.99,5000,0,Entertainment,0,3.0,39,5,10,0,Unsecured
3,4,Blue Cash Preferred Card from Express,11.99,5000,0,Entertainment,0,0.0,39,3,10,0,Unsecured
4,5,Customized Cash Rewards credit card,13.99,5000,0,Entertainment,0,3.0,39,3,10,0,Unsecured


In [7]:
upload_df = pd.read_csv('/data/train_df_latest.csv')

In [8]:
upload_df.columns

Index(['CUSTOMER_ID', 'AGE', 'EMPLOYMENT_STATUS', 'HOBBIES', 'MARITAL_STATUS',
       'BANK_ACCOUNT_TYPE', 'BALANCE', 'CREDIT_LIMIT', 'CASH_ADVANCE',
       'PURCHASES', 'INSTALLMENTS_PURCHASES', 'PAYMENTS', 'CLUSTER_TYPE',
       'HIGHEST_SPENT', 'CARD_ID', 'CARD_NAME', 'CARD_INTERESTFEE',
       'CARD_CREDITLIMIT', 'CARD_ANNUALFEE', 'CARD_LATEFEE', 'CARD_BALANCEFEE',
       'CARD_ADVANCEFEE'],
      dtype='object')

In [9]:
upload_df.head()

,CUSTOMER_ID,AGE,EMPLOYMENT_STATUS,HOBBIES,MARITAL_STATUS,BANK_ACCOUNT_TYPE,BALANCE,CREDIT_LIMIT,CASH_ADVANCE,PURCHASES,...,CLUSTER_TYPE,HIGHEST_SPENT,CARD_ID,CARD_NAME,CARD_INTERESTFEE,CARD_CREDITLIMIT,CARD_ANNUALFEE,CARD_LATEFEE,CARD_BALANCEFEE,CARD_ADVANCEFEE
0,PNC31d3482a-ba31-42c4-aa7a-27b9d35c3,66,Retired,Undisclosed,Widowed,Business,7492.531898,9025.0,3352.380760,1036.0382,...,Cash Advance Users,Cash Transactions,21,City Miscellaneous Card,14.99,7000,0,37,4,10
1,PNC28078b5c-7d7e-4d5d-852d-743557151,46,Unemployed,Undisclosed,Single,Business,136.920751,15345.0,43.695068,3115.6536,...,Active Credit Users,Health care,21,City Miscellaneous Card,14.99,7000,0,37,4,10
2,PNCea940b80-c329-4151-840a-25e1b1152,46,Unemployed,Undisclosed,Single,Business,154.035845,15015.0,43.220121,3589.7748,...,Active Credit Users,Travel,18,The Platinum Card from Express,0.00,0,695,0,0,0
3,PNC1031148a-9223-4144-bc99-2000bfdac,25,Unemployed,Undisclosed,Single,Checking,163.087413,6580.0,0.000000,468.5692,...,Conservative Spenders,Health care,21,City Miscellaneous Card,14.99,7000,0,37,4,10
4,PNC60f353b5-5c34-459f-8766-956fe378b,25,Unemployed,Undisclosed,Single,Checking,193.223130,7245.0,0.000000,468.5692,...,Conservative Spenders,Health care,21,City Miscellaneous Card,14.99,7000,0,37,4,10


In [10]:
#Import all snowflake connection details from Template or Project variables.

db_user = os.getenv('Snowflake_user')
db_password =  os.getenv('Snowflake_password')
db_account = os.getenv('Snowflake_Account')
db_database =  os.getenv('Snowflake_Database')
db_role = os.getenv('Snowflake_user')
db_warehouse = os.getenv('Snowflake_Warehouse')
db_schema = os.getenv('Snowflake_Schema')

In [11]:
from snowflake.snowpark.session import Session
connection_params = {
    'user': db_user,
    'password': db_password,
    'account': db_account,
    'warehouse': db_warehouse,
    'database': db_database,
    'schema': db_schema,
    'role': db_role
}
session = Session.builder.configs(connection_params).create()

In [12]:
df_train_sf=session.createDataFrame(
        upload_df.values.tolist(),
        schema=upload_df.columns.tolist())
df_train_sf.write.mode("overwrite").save_as_table("FDC_Insurance.INS_CUST_SEGMT_SCHEMA.CUSTOMER_DATA_INSIGHT_TRAIN_CATEGORIZE")

In [13]:
cc_train_sf=session.createDataFrame(
        cc_df.values.tolist(),
        schema=cc_df.columns.tolist())
cc_train_sf.write.mode("overwrite").save_as_table("FDC_Insurance.INS_CUST_SEGMT_SCHEMA.CREDIT_CARD_DETAILS")

### JOIN CUSTOMER_DATA_INSIGHT_TRAIN_CATEGORIZE WITH CUSTOMER_DATA_INSIGHT_TRAIN_OUTPUT to Add newly added column

ALTER TABLE FDC_INSURANCE.INS_CUST_SEGMT_SCHEMA.CUSTOMER_DATA_INSIGHT_TRAIN_OUTPUT ADD COLUMN HIGHEST_SPENT VARCHAR;

UPDATE FDC_INSURANCE.INS_CUST_SEGMT_SCHEMA.CUSTOMER_DATA_INSIGHT_TRAIN_OUTPUT A
SET A.HIGHEST_SPENT=B.HIGHEST_SPENT
FROM FDC_INSURANCE.INS_CUST_SEGMT_SCHEMA.CUSTOMER_DATA_INSIGHT_TRAIN_CATEGORIZE B
WHERE A.CUSTOMER_ID=B.CUSTOMER_ID